In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [26]:
def get_links(page_link):
    links=[]
    r = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    for ana in soup.findAll('a'):
        if ana.parent.name == 'h2':
            links.append(ana['href'])
    return links[1:]

def get_text(article_link):
    article=""
    r = requests.get(article_link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    soup_article=soup.find("article")
    [s.extract() for s in soup_article('blockquote')]

    #get author
    #get publication date
    #get content
    for t in soup_article.findAll('p'):
        excerpt = t.text.replace(u'\xa0', u' ')
        if "[[{“type”:”media”" not in excerpt:
            article = article + "¶" + excerpt
    return article

def get_title(article_link):
    r = requests.get(article_link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    soup_article=soup.find("article")
    [s.extract() for s in soup_article('blockquote')]
    #get title
    title = soup_article.find('h1', {'class':'post-title'})
    if not title:
        print("No title for {}".format(article_link))
    return title.text

def get_author(article_link):
    r = requests.get(article_link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    soup_article=soup.find("article")
    [s.extract() for s in soup_article('blockquote')]
    #get author
    author = soup_article.find('a', {'class':'author'})
    if not author:
        print("No title for {}".format(article_link))
    return author.text

def get_date(article_link):
    r = requests.get(article_link, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    soup_article=soup.find("article")
    [s.extract() for s in soup_article('blockquote')]
    #get author
    date = soup_article.find('time', {'class':'entry-date'})
    if not date:
        print("No title for {}".format(article_link))
    return date.text

In [27]:
headers = {'User-Agent':'Mozilla/5.0'}
alternet_titles=[]
alternet_authors=[]
alternet_dates=[]
alternet_articles=[]
for i in range(100):
    links = get_links("https://www.alternet.org/tag/climate-change/page/{}/".format(i))
    for link in links:
        alternet_articles.append(get_text(link))
        alternet_titles.append(get_title(link))
        alternet_dates.append(get_date(link))
        alternet_authors.append(get_author(link))

In [28]:
len(alternet_titles)

1000

In [29]:
df = pd.DataFrame(alternet_articles, columns=['content'])
df['title']=alternet_titles
df['author']=alternet_authors
df['date']=alternet_dates
df['denial?'] = [0]*len(df.index)
cols=['title','author','date','content','denial?']
df=df[cols]
df = df.drop_duplicates()
df.to_csv('../../data/left/data_alternet_articles.csv', index=False)

In [30]:
df.shape

(990, 5)